# From text to eKG

This notebook has the minimum sample code to convert a transcribed interaction into is graph representation.

In [1]:
# Imports
from datetime import date
from pathlib import Path
from random import getrandbits

import requests
from cltl.brain.long_term_memory import LongTermMemory
from cltl.entity_linking.label_linker import LabelBasedLinker
from cltl.triple_extraction.api import Chat
# from cltl.triple_extraction.cfg_analyzer import CFGAnalyzer
from cltl.triple_extraction.spacy_analyzer import spacyAnalyzer
from cltl.triple_extraction.utils.helper_functions import utterance_to_capsules
from tqdm import tqdm

In [2]:
# Data
baking_scenario = [
    "Selene: I like baking cakes",
    "Lea: I don't like cakes",
    "Selene: I also like baking cookies",
    "Lea: I like cookies",
    "Selene: do you like chocolate cookies?",
    "Lea: I love chocolate cookies"
    "Selene: I will bake chocolate cookies tonight"
]
baking_scenario_speakers = ["Selene", "Lea"]

In [3]:
def create_context_capsule():
    # Define contextual features
    context_id = getrandbits(8)
    place_id = getrandbits(8)
    location = requests.get("https://ipinfo.io").json()
    start_date = date(2021, 3, 12)

    return {"context_id": context_id,
            "date": start_date,
            "place": "Unknown",
            "place_id": place_id,
            "country": location['country'],
            "region": location['region'],
            "city": location['city']}

In [8]:
def main(scenario, speakers):
    # Set logging levels
    #     chat_logger.setLevel(logging.ERROR)
    #     brain_logger.setLevel(logging.ERROR)

    # Create analyzers
    analyzer = spacyAnalyzer()
    linker = LabelBasedLinker()

    # Create folders
    graph_filepath = Path('./graph/baking_graph/')

    # Initialize brain, Chat, 
    brain = LongTermMemory(address="http://localhost:7200/repositories/sandbox",  # Location to save accumulated graph
                           log_dir=graph_filepath,  # Location to save step-wise graphs
                           clear_all=True)  # To start from an empty brain
    chat = Chat(speakers[0], speakers[-1])

    # Create context
    context_capsule = create_context_capsule()
    brain.capsule_context(context_capsule)

    all_capsules = []
    capsules_skipped = 0
    for turn, utterance in tqdm(enumerate(scenario)):
        # split speaker from utterance
        speaker, utterance = utterance.split(': ')
        listeners = list(filter(lambda x: x != speaker, speakers))

        # add utterance to chat and use spacy analyzer to analyze
        chat.add_utterance(utterance, speaker)
        analyzer.analyze(chat.last_utterance, speaker, listeners[0])
        capsules = utterance_to_capsules(chat.last_utterance)

        # add statement capsules to brain
        for capsule in capsules:
            print(capsule)
            linker.link(capsule)

            try:
                # Add capsule to brain
                print("\tAdding capsule to brain")
                response = brain.capsule_statement(capsule)
                row[key][i]['rdf_file'].append(str(response['rdf_log_path'].stem) + '.trig')
            except:
                capsules_skipped += 1
                print(f"\tCapsule skipped. Total skipped: {capsules_skipped}")





In [9]:
main(baking_scenario, baking_scenario_speakers)

2023-05-25 22:33:55 -     INFO -                                    cltl.brain.LongTermMemory - Booted
2023-05-25 22:33:55 -     INFO -                                    cltl.brain.LongTermMemory - Clearing brain
2023-05-25 22:33:55 -     INFO -                                    cltl.brain.LongTermMemory - Uploading ontology to brain
2023-05-25 22:33:56 -     INFO -                                  cltl.brain.ThoughtGenerator - Booted
2023-05-25 22:33:56 -     INFO -                                  cltl.brain.LocationReasoner - Booted
2023-05-25 22:33:56 -     INFO -                                      cltl.brain.TypeReasoner - Booted
2023-05-25 22:33:56 -     INFO -                                   cltl.brain.TrustCalculator - Booted
2023-05-25 22:33:56 -     INFO -                                  cltl.triple_extraction.Chat - << Start of Chat with Lea >>
2023-05-25 22:33:56 -     INFO -                                    cltl.brain.LongTermMemory - Context: context200
0it [00:0

{'chat': 50, 'turn': 1, 'author': 'Lea', 'utterance': "I don't like cakes", 'utterance_type': <UtteranceType.STATEMENT: '1'>, 'position': '0-18', 'subject': {'label': 'Lea', 'type': []}, 'predicate': {'label': 'like', 'type': []}, 'object': {'label': 'cake', 'type': []}, 'perspective': {}, 'context_id': None, 'date': '2023-05-25T22:33:56.918334', 'place': '', 'place_id': None, 'country': '', 'region': '', 'city': '', 'objects': [], 'people': []}
{'label': 'Lea', 'type': []}
{'label': 'cake', 'type': []}
Lea


AttributeError: 'str' object has no attribute 'keys'